# Chinese NER Test Pipeline 
## By: Cameron B.

### Download model from ModelScope

In [1]:
from modelscope.hub.snapshot_download import snapshot_download

# load model (only done first time)
model_dir = snapshot_download('iic/nlp_raner_named-entity-recognition_chinese-base-generic', cache_dir='/scratch/ssd004/scratch/cambish/CNER')

2024-10-10 14:22:23,904 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


### Create NER pipeline and load data

In [2]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.models import Model

from datasets import load_dataset

# set up tokenization from https://huggingface.co/docs/datasets/en/use_dataset?
# model seems to work fine without specifying tokenizer

# tokenizer = BertTokenizer.from_pretrained()
cfp = load_dataset('/scratch/ssd004/scratch/cambish/cmmlu-v1.0.1/cmmlu.py', "chinese_foreign_policy", split="dev")

ner_model = Model.from_pretrained(model_dir)
ner_pipeline = pipeline(Tasks.named_entity_recognition, model=ner_model)

# test sample
ner_input = ['尼克松在回顾1972年访华时说：“我知道，1954年在……时福斯特·杜勒斯（当时的美国国务卿）拒绝同周（周恩来）握手，使他身受侮辱。因此我走完梯级时决心伸出我的手，一边向他走去。当我们的手相握的时候，一个时代结束了，另一个时代开始了。”对上述材料理解正确的是',
            '尼克松与周恩来握手意味着美国彻底放弃了遏制中国政策', '尼克松否定了福斯特·杜勒斯的对华态度', '中美关系对20世纪70年代的国际关系有重要影响', '省略部分应是万隆会议']
outputs = ner_pipeline(ner_input)
print(outputs)
    

2024-10-10 14:22:31,334 - modelscope - INFO - initialize model from /scratch/ssd004/scratch/cambish/CNER/iic/nlp_raner_named-entity-recognition_chinese-base-generic
2024-10-10 14:22:38,720 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
/h/cambish/.conda/envs/modelscope/lib/python3.9/site-packages/modelscope/utils/checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.a

[{'output': [{'type': 'PER', 'start': np.int64(0), 'end': np.int64(3), 'prob': np.float32(0.6087498), 'span': '尼克松'}, {'type': 'GPE', 'start': np.int64(12), 'end': np.int64(13), 'prob': np.float32(0.9466083), 'span': '华'}, {'type': 'PER', 'start': np.int64(30), 'end': np.int64(37), 'prob': np.float32(0.6117986), 'span': '福斯特·杜勒斯'}, {'type': 'GPE', 'start': np.int64(41), 'end': np.int64(43), 'prob': np.float32(0.6031834), 'span': '美国'}, {'type': 'PER', 'start': np.int64(50), 'end': np.int64(51), 'prob': np.float32(0.93963814), 'span': '周'}, {'type': 'PER', 'start': np.int64(52), 'end': np.int64(55), 'prob': np.float32(0.43081495), 'span': '周恩来'}]}, {'output': [{'type': 'PER', 'start': np.int64(0), 'end': np.int64(3), 'prob': np.float32(0.64598316), 'span': '尼克松'}, {'type': 'PER', 'start': np.int64(4), 'end': np.int64(7), 'prob': np.float32(0.32906184), 'span': '周恩来'}, {'type': 'GPE', 'start': np.int64(12), 'end': np.int64(14), 'prob': np.float32(0.58992296), 'span': '美国'}, {'type': 'GPE

### Visualize Output w/ SpaCy

In [4]:
import spacy
from spacy.tokens import DocBin
from spacy import displacy

nlp = spacy.blank("zh")

# docs list will store annotations
docs = []
for text, annotations in zip(ner_input, outputs):
    doc = nlp(text)
    ents = []
    # required to convert annotations dict of lists into annotation dict
    for annotation in annotations['output']:
        span = doc.char_span(annotation['start'], annotation['end'], label=annotation['type'])
        ents.append(span)
    doc.ents = ents
    docs.append(doc)

displacy.render(docs, style="ent")


/h/cambish/.conda/envs/modelscope/lib/python3.9/site-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


OSError: Can't load tokenizer for './'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.